# OOP I: CLASS VS OBJECT

Вашей задачей является создание класса Python для обработки данных DataProcessor. Вам следует определить атрибуты, такие как data и processed_data_.

Напишите метод под названием process, который не принимает параметры. Этот метод должен реализовать логику предобработки данных и сохранить результат в атрибут processed_data_.

Предобработка следующая: Необходимо вычислить отклонение от среднего. Для всего набора данных data вычислить среднее арифметическое и для каждого значения вычислить x_i  − x_mean и сохранить в processsed_data_.

Напишите метод save_to_file, который принимает имя файла в качестве параметра и сохраняет обработанные данные в указанный файл. Каждое значение в списке processed_data_ необходимо сохранять на отдельной строке. Если данные не были обработаны, файл сохранять не нужно.

In [ ]:
class DataProcessor:
    """DataProcessor implementation"""

    def __init__(self, data):
        self.data = data
        self.processed_data_ = None

    def process(self):
        """Process the data"""
        if not self.data:
            return
        x_mean = sum(self.data) / len(self.data)
        self.processed_data_ = [x - x_mean for x in self.data]

    def save_to_file(self, filename):
        """Save the data to a file"""
        if self.processed_data_ is None:
            return

        with open(filename, 'w') as f:
            for data_point in self.processed_data_:
                f.write(f"{data_point}\n")


# OOP II: PRINCIPLES

Принцип инкапсуляции
Принцип заключается в том, что внутреннее представление объекта скрыто от внешнего мира, а доступ к данным и методам объекта осуществляется только через установленные интерфейсы.
Принцип наследования
Принцип заключается в том, что парадигма ООП позволяет создавать новые классы на основе существующих, наследуя их свойства и методы. Рассмотрим снова пример с градиентным бустингом из Scikit-learn.
Принцип полиморфизма
Принцип говорит о том, что парадигма ООП позволяет одинаковые методы в разных классах с различной реализацией. Еще говорят, что полиморфизм позволяет использовать общий интерфейс.

Или рассмотрим другой пример, в котором мы хотим реализовать несколько классов для валидации в нашем проекте. Класс KFoldValidation, который выполняет кросс-валидацию, и RepeatedKFoldValidation, который выполняет кросс-валидацию с повторением.

Оба класса будут иметь часть общей логики и часть собственного поведения. Поэтому удобно создать единый класс-родитель и реализовать в нем общую логику. А сами классы наследовать от родительского и добавить в них специфичное поведение.

In [3]:
from sklearn.model_selection import KFold, RepeatedKFold


class BaseValidation:
    def __init__(self, n_splits):
        self.n_splits = n_splits
        self.validator = None

    def split_data(self, data):
        # Общий метод для разделения данных для валидации
        return self.validator.split(data)


class KFoldValidation(BaseValidation):
    def __init__(self, n_splits):
        super().__init__(n_splits)
        self.validator = KFold(n_splits=self.n_splits)

    def perform_validation(self, model, data):
        # Метод для выполнения кросс-валидации
        for train_index, test_index in self.split_data(data):
            train_data, test_data = data[train_index], data[test_index]
            model.fit(train_data)
            accuracy = model.evaluate(test_data)
            print(f"KFold Validation Accuracy: {accuracy}")

class RepeatedKFoldValidation(BaseValidation):
    def __init__(self, n_splits, n_repeats):
        super().__init__(n_splits)
        self.n_repeats = n_repeats
        self.validator = RepeatedKFold(n_splits=self.n_splits, n_repeats=self.n_repeats)

    def perform_validation(self, model, data):
        # Метод для выполнения кросс-валидации с повторением
        for train_index, test_index in self.split_data(data):
            train_data, test_data = data[train_index], data[test_index]
            model.fit(train_data)
            accuracy = model.evaluate(test_data)
            print(f"RepeatedKFold Validation Accuracy: {accuracy}")

В этом примере класс BaseValidation служит базовым классом, который имеет общий метод для разделения данных для валидации (split_data). Дочерние классы KFoldValidation и RepeatedKFoldValidation добавляют свои собственные методы для выполнения кросс-валидации на основе соответствующих стратегий разделения данных (KFold и RepeatedKFold соответственно).

Одно из проявлений полиморфизма в Python - это возможность переопределять поведение методов в дочерних классах.

Абстрактные классы - это классы, которые содержат абстрактные методы, то есть методы без конкретной реализации в самом классе. Они служат в качестве шаблона для дочерних классов, которые обязаны реализовать абстрактные методы.

Абстрактные классы позволяют создавать интерфейсы - соглашение о том какие методы должны быть у дочерних классов.  Например, мы договариваемся, что все классы-коннекторы к базам данных будут содержать методы connect, fetch, insert. Мы можем создать соответствующий абстрактныйкласс DBConnector, в котором описать как должны называться эти методы, какие аргументы принимать и что возвращать. А затем реализовать несколько конкретных реализаций этого абстрактного класса, унаследовавшись от него: MySQLConnector, PostgresConnector, OracleConnector.

In [4]:
from abc import ABC, abstractmethod

class MLModel(ABC):
    @abstractmethod
    def fit(self, X_train, y_train):
        pass

    @abstractmethod
    def predict(self, X_test):
        pass

class LinearModel(MLModel):
    def fit(self, X_train, y_train):
        print("Training Linear Model")

    def predict(self, X_test):
        print("Predicting with Linear Model")

class RandomForestModel(MLModel):
    def fit(self, X_train, y_train):
        print("Training Random Forest Model")

    def predict(self, X_test):
        print("Predicting with Random Forest Model")

# Пример использования полиморфизма
def train_and_predict(model, X_train, y_train, X_test):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    return predictions

# Создаем экземпляры разных моделей
linear_model = LinearModel()
random_forest_model = RandomForestModel()

# Производим обучение и предсказание с использованием полиморфизма
X_train, y_train, X_test = ..., ..., ...  # Замените на реальные данные
predictions_linear = train_and_predict(linear_model, X_train, y_train, X_test)
predictions_rf = train_and_predict(random_forest_model, X_train, y_train, X_test)


Training Linear Model
Predicting with Linear Model
Training Random Forest Model
Predicting with Random Forest Model


# Задание

## UniqueColumnSampler
Есть базовый класс RandomSampler, реализующий случайное семплирование данных из датафрейма. Вам необходимо создать дочерний класс UniqueRandomСolumnSampler, который наследуется от RandomSampler и переопределяет его функциональность. Дочерний класс должен иметь возможность выбора уникальных значений из указанного столбца.

Ожидаемое поведение:
- RandomSampler уже реализован и использует встроенный метод sample для случайного семплирования данных из датафрейма
- UniqueColumnSampler должен выбирать уникальные значения из указанного столбца и возвращать новый датафрейм с этими значениями. Результирующий датафрейм содержит только одну колонку.

In [5]:
import random
import pandas as pd


class RandomSampler:
    """ "Randomly samples data from a dataframe."""

    def __init__(self, dataframe):
        self.dataframe = dataframe

    def get_num_rows(self):
        """Returns the number of rows in the dataframe."""
        return len(self.dataframe)

    def sample_data(self, num_samples):
        """Samples a specified number of rows from the dataframe."""
        return self.dataframe.sample(num_samples)


class UniqueColumnSampler(RandomSampler):
    """Samples unique values from a specified column in the dataframe."""

    def __init__(self, dataframe, column_name):
        super().__init__(dataframe)
        self.column_name = column_name

    def sample_data(self, num_samples):
        """Samples unique values from the specified column."""
        unique_values = self.dataframe[self.column_name].unique()
        sampled_values = random.sample(list(unique_values), min(num_samples, len(unique_values)))
        return pd.DataFrame({self.column_name: sampled_values})

## DataPreprocessor
Вы с коллегой разрабатываете пайплайн предобработки данных. Вы договорились, что в пайплайне могут использоваться разные методы обработки данных. Все эти методы объединяет то, что:

Они принимают на вход список числовых значений
Возвращают список числовых значений
Вы договорились, что каждый метод обработки будет представлять собой отдельный класс. Чтобы обеспечить единообразное поведение этих классов, вы договорились что все они будут реализовывать единый интерфейс (абстрактный класс) DataPreprocessor, который содержит единственный метод preprocess. Метод принимает список чисел и возвращает обработанный список чисел.

Ваша задача
Реализовать абстрактный класс DataPreprocessor с единственным методом preprocess. Метод принимает на вход аргумент data.
Реализовать 3 дочерних классах OutlierRemover, Normalizer, Encoder. В каждом классе реализуется собственная логика метода preprocess.

Класс OutlierRemover должен удалять выбросы из данных. Реализуйте метод preprocess, который принимает на вход данные и возвращает данные без выбросов. Просто удалите все значения, превышающие 10 из списка.
Класс Normalizer должен нормализовать входные данные. Реализуйте метод preprocess, который принимает на вход данные и возвращает данные с нормализованные данные. Просто разделите каждое значение на 10.
Класс Encoder должен кодировать категориальные признаки. Реализуйте метод preprocess, который принимает на вход данные и возвращает данные с закодированными категориальными признаками. Для этого используйте словарь для кодировки. Key в словаре исходное незакодированное значение. Value - закодированное значение. Предполагаем, что все необходимые пары key-value содержатся в словаре.
Обратите внимание, что у класса Encoder необходимо  определить конструктор, в который передается словарь для кодировки.

In [25]:
from abc import ABC, abstractmethod


class DataPreprocessor(ABC):
    """Data preprocessor class"""

    @abstractmethod
    def preprocess(self, data):
        pass
    
    
class OutlierRemover(DataPreprocessor):
    """Outlier removal algorithm"""

    def preprocess(self, data):
        return list(filter(lambda x: x < 10, data))


class Normalizer(DataPreprocessor):
    """Normalizer for data"""

    def preprocess(self, data):
        return list(map(lambda x: x/10, data))
    
    
class Encoder(DataPreprocessor):
    """Encoder class for encoding data"""
    def __init__(self, encoding_dict=None):
        self.encoding_dict = encoding_dict or {}

    def preprocess(self, data):
        if data is None:
            return []
        
        return [self.encoding_dict.get(value, 0) for value in data]
        

Класс Encoder должен кодировать категориальные признаки. Реализуйте метод preprocess, который принимает на вход данные и возвращает данные с закодированными категориальными признаками. Для этого используйте словарь для кодировки. Key в словаре исходное незакодированное значение. Value - закодированное значение. Предполагаем, что все необходимые пары key-value содержатся в словаре.

In [26]:
# Пример использования OutlierRemover
outlier_remover = OutlierRemover()
data_with_outliers = [1, 2, 3, 100, 5, 6, 7, 8, 9]
cleaned_data = outlier_remover.preprocess(data_with_outliers)

print(f"Исходные данные: {data_with_outliers}")
print(f"Данные без выбросов: {cleaned_data}")

Исходные данные: [1, 2, 3, 100, 5, 6, 7, 8, 9]
Данные без выбросов: [1, 2, 3, 5, 6, 7, 8, 9]


In [27]:
# Пример использования Normalizer
normalizer = Normalizer()

numerical_data = [10, 20, 30, 40, 50]
normalized_data = normalizer.preprocess(numerical_data)

print(f"Исходные числовые данные: {numerical_data}")
print(f"Нормализованные данные: {normalized_data}")

Исходные числовые данные: [10, 20, 30, 40, 50]
Нормализованные данные: [1.0, 2.0, 3.0, 4.0, 5.0]


In [28]:
# Пример использования Encoder
encoder = Encoder(encoding_dict={'красный': 1, 'зеленый': 2, 'синий': 3})

categorical_data = ['красный', 'зеленый', 'синий']
encoded_data = encoder.preprocess(categorical_data)

print(f"Исходные категориальные данные: {categorical_data}")
print(f"Закодированные данные: {encoded_data}")


Исходные категориальные данные: ['красный', 'зеленый', 'синий']
Закодированные данные: [1, 2, 3]
